# Stochastique Project

Colonne	Description
lead_id	ID du numéro appelé (le prospect)
date_a{k}	Date à laquelle le prospect (lead_id) a été contacté lors de la tentative k
duration_a{k}	Durée de l'appel lors de la tentative k
caller_id_a{k}	ID du numéro utilisé pour appeler le prospect (lead_id) lors de la tentative k
lead_answered_a{k}	Indique si le prospect (lead_id) a répondu lors de la tentative k
caller_has_changed_a{k}	Indique si le numéro appelant (caller_id) a changé entre la tentative k-1 et k

In [1]:
# code permettant de charger tous les fichiers de données et de les stocker dans un dictionnaire

import pandas as pd
import tabulate

calls_dfd = {}
for k in range(1, 5):
    data_filename = f"calls_a{k}.csv.bz2"
    print(f"Chargement du fichier {data_filename}")
    calls_dfd[f"a{k}"] = pd.read_csv(
        data_filename,
        parse_dates=[f"date_a{k_bis}" for k_bis in range(1, k + 1)],
    )

Chargement du fichier calls_a1.csv.bz2
Chargement du fichier calls_a2.csv.bz2
Chargement du fichier calls_a3.csv.bz2
Chargement du fichier calls_a4.csv.bz2


In [2]:
import tabulate

# Affichage des premiers numéros ayant été contactés lors de la 2ème tentative :
print(tabulate.tabulate(calls_dfd["a2"].head(5), tablefmt="orgtbl", headers="keys", showindex=False))


| lead_id      | date_a1             | weekday_a1   | hour_a1   |   duration_a1 | caller_id_a1   | lead_answered_a1   | date_a2             | weekday_a2   | hour_a2   |   duration_a2 | caller_id_a2   | lead_answered_a2   | caller_has_changed_a2   |
|--------------+---------------------+--------------+-----------+---------------+----------------+--------------------+---------------------+--------------+-----------+---------------+----------------+--------------------+-------------------------|
| fdeae76d3fef | 2020-04-03 10:05:49 | 0Monday      | h10       |             2 | 1e540d0ac463   | no                 | 2020-04-06 18:28:15 | 3Thursday    | h18       |             2 | 08fcb33240a2   | no                 | yes                     |
| 73c015525161 | 2020-04-03 10:05:50 | 0Monday      | h10       |             2 | 159e9c0a04f6   | no                 | 2020-04-06 18:28:15 | 3Thursday    | h18       |             2 | f986b4d3e9dd   | no                 | yes                     |
| c7

In [3]:
# Analyse des variables catégorielles pour les données de la 2ème tentative.

tabulate.tabulate(calls_dfd["a2"].describe(include="O"), tablefmt="orgtbl", headers="keys")

'|        | lead_id      | weekday_a1   | hour_a1   | caller_id_a1   | lead_answered_a1   | weekday_a2   | hour_a2   | caller_id_a2   | lead_answered_a2   | caller_has_changed_a2   |\n|--------+--------------+--------------+-----------+----------------+--------------------+--------------+-----------+----------------+--------------------+-------------------------|\n| count  | 2824096      | 2824096      | 2824096   | 2824096        | 2824096            | 2824096      | 2824096   | 2824096        | 2824096            | 2824096                 |\n| unique | 2761053      | 5            | 9         | 900            | 1                  | 5            | 9         | 987            | 2                  | 2                       |\n| top    | 40f09fe86afe | 2Wednesday   | h11       | 094fd601829a   | no                 | 3Thursday    | h11       | 094fd601829a   | no                 | yes                     |\n| freq   | 3            | 620362       | 371260    | 56839          | 2824096       

In [4]:
# Analyse des numéros qui ont décroché à la première tentative.

calls_dfd["a1"]["lead_answered_a1"].value_counts()

lead_answered_a1
no     4847340
yes    1465250
Name: count, dtype: int64

In [5]:
# Récupération du nombre de numéros appelés lors de la deuxième tentative :

len(calls_dfd["a2"])

2824096

In [6]:
# Préparation des données
# On change le type des variables catégorielles avec la méthode .astype('category') :

var_cat = ["weekday_a1", "hour_a1", "weekday_a2", "hour_a2", "caller_has_changed_a2", "lead_answered_a2"]
for var in var_cat:
    calls_dfd["a2"][var] = calls_dfd["a2"][var].astype('category')

calls_dfd["a2"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2824096 entries, 0 to 2824095
Data columns (total 14 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   lead_id                object        
 1   date_a1                datetime64[ns]
 2   weekday_a1             category      
 3   hour_a1                category      
 4   duration_a1            int64         
 5   caller_id_a1           object        
 6   lead_answered_a1       object        
 7   date_a2                datetime64[ns]
 8   weekday_a2             category      
 9   hour_a2                category      
 10  duration_a2            int64         
 11  caller_id_a2           object        
 12  lead_answered_a2       category      
 13  caller_has_changed_a2  category      
dtypes: category(6), datetime64[ns](2), int64(2), object(4)
memory usage: 188.5+ MB


In [7]:
# Création d'un premier modèle
# Nous proposons de créer un modèle permettant d'expliquer la variable lead_answered_a2 à partir de hour_a1 et hour_a2.

# On commence par créer les variables du réseau.

import pyAgrum as gum

var_to_model = ["hour_a1", "hour_a2", "lead_answered_a2"]

var_bn = {}
for var in var_to_model:
    nb_values = len(calls_dfd["a2"][var].cat.categories)
    var_bn[var] = gum.LabelizedVariable(var, var, nb_values)

# On renseigne les modalités des variables pour être fidèle à celles des données.

for var in var_bn:
    for i, modalite in enumerate(calls_dfd["a2"][var].cat.categories):
        var_bn[var].changeLabel(i, modalite)
# Ensuite, on crée le réseau bayésien et on ajoute les variables sélectionnées.

bn = gum.BayesNet("modèle simple")

for var in var_bn.values():
    bn.add(var)
# Pour finir, on fixe la structure.

bn.addArc("hour_a1", "lead_answered_a2")
bn.addArc("hour_a2", "lead_answered_a2")

In [8]:
# PC de hour_a1 :

bn.cpt("hour_a1")

(pyAgrum.Potential<double>@00000250E04F9780) 
  hour_a1                                                                                |
h10      |h11      |h12      |h14      |h15      |h16      |h17      |h18      |h19      |
---------|---------|---------|---------|---------|---------|---------|---------|---------|
 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  |

In [9]:
# LPC de hour_a2 :

bn.cpt("hour_a2")

(pyAgrum.Potential<double>@00000250E04F9B80) 
  hour_a2                                                                                |
h10      |h11      |h12      |h14      |h15      |h16      |h17      |h18      |h19      |
---------|---------|---------|---------|---------|---------|---------|---------|---------|
 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  |

In [10]:
# LPC de probabilité de lead_answered_a2 | hour_a1, hour_a2 :

bn.cpt("lead_answered_a2")

# On remarque naturellement que les LPC ne contiennent pour l'instant que des zéros…

(pyAgrum.Potential<double>@00000250E04F9980) 
             ||  lead_answered_a2 |
hour_a|hour_a||no       |yes      |
------|------||---------|---------|
h10   |h10   || 0.0000  | 0.0000  |
h11   |h10   || 0.0000  | 0.0000  |
h12   |h10   || 0.0000  | 0.0000  |
h14   |h10   || 0.0000  | 0.0000  |
h15   |h10   || 0.0000  | 0.0000  |
h16   |h10   || 0.0000  | 0.0000  |
[...69 more line(s) ...]
h14   |h19   || 0.0000  | 0.0000  |
h15   |h19   || 0.0000  | 0.0000  |
h16   |h19   || 0.0000  | 0.0000  |
h17   |h19   || 0.0000  | 0.0000  |
h18   |h19   || 0.0000  | 0.0000  |
h19   |h19   || 0.0000  | 0.0000  |

In [11]:
# Apprentissage des LPC
# Pour apprendre les paramètres des LPC à partir des données, vous pouvez utiliser la classe gum.BNLearner et en particulier la méthode fitParameters:

learner = gum.BNLearner(calls_dfd["a2"][var_to_model])

learner.fitParameters(bn)

(pyAgrum.BNLearner<double>@00000250A669A030) Filename       : C:\Users\pierr\AppData\Local\Temp\tmpl62b3ea2.csv
Size           : (2824096,3)
Variables      : hour_a1[9], hour_a2[9], lead_answered_a2[2]
Induced types  : True
Missing values : False
Algorithm      : MIIC
Score          : BDeu  (Not used for constraint-based algorithms)
Correction     : MDL  (Not used for score-based algorithms)
Prior          : -

In [12]:
# Après apprentissage, nous obtenons les LPC suivantes. LPC de hour_a1 :

bn.cpt("hour_a1")

(pyAgrum.Potential<double>@00000250E04F9780) 
  hour_a1                                                                                |
h10      |h11      |h12      |h14      |h15      |h16      |h17      |h18      |h19      |
---------|---------|---------|---------|---------|---------|---------|---------|---------|
 0.1176  | 0.1315  | 0.1298  | 0.0880  | 0.1178  | 0.1182  | 0.0844  | 0.1067  | 0.1059  |

In [13]:
# LPC de hour_a2 :

bn.cpt("hour_a2")

(pyAgrum.Potential<double>@00000250E04F9B80) 
  hour_a2                                                                                |
h10      |h11      |h12      |h14      |h15      |h16      |h17      |h18      |h19      |
---------|---------|---------|---------|---------|---------|---------|---------|---------|
 0.1027  | 0.1372  | 0.1226  | 0.0868  | 0.1194  | 0.1194  | 0.0800  | 0.1136  | 0.1184  |

In [14]:
# LPC de probabilité de lead_answered_a2 | hour_a1, hour_a2 :

bn.cpt("lead_answered_a2")

(pyAgrum.Potential<double>@00000250E04F9980) 
             ||  lead_answered_a2 |
hour_a|hour_a||no       |yes      |
------|------||---------|---------|
h10   |h10   || 0.8598  | 0.1402  |
h11   |h10   || 0.8508  | 0.1492  |
h12   |h10   || 0.8558  | 0.1442  |
h14   |h10   || 0.8429  | 0.1571  |
h15   |h10   || 0.8485  | 0.1515  |
h16   |h10   || 0.8486  | 0.1514  |
[...69 more line(s) ...]
h14   |h19   || 0.8367  | 0.1633  |
h15   |h19   || 0.8342  | 0.1658  |
h16   |h19   || 0.8396  | 0.1604  |
h17   |h19   || 0.8367  | 0.1633  |
h18   |h19   || 0.8521  | 0.1479  |
h19   |h19   || 0.8548  | 0.1452  |

In [15]:
import pyAgrum as gum
import pandas as pd
import numpy as np
import sys

def predict(bn, data, var_target, show_progress=False):
    """
    This function is used to predict the posterior probability of a target variable from observations  
    using a bayesian network model. 

    Inputs:
    - =bn=: the predictive model given as a =pyAgrum.BayesNet= object
    - =data=: the data containing the observations used to predict the target variable 
    as a =pandas.DataFrame= object
    - =var_target=: the name of the target variable as a =str= object

    Returns:
    - a =DataFrame= containing the posterior probability distribution of the target 
    variable given each observation in =data=.
    """
    # Initialize the inference engine
    inf_bn = gum.LazyPropagation(bn)
    inf_bn.setTargets({var_target})
    nb_data = len(data)
    target_size = bn.variable(var_target).domainSize()
    target_dom = np.array([bn.variable(var_target).label(i)
                           for i in range(target_size)])
    data_records = data.to_dict("records")
    post_prob = np.zeros((nb_data, target_size))
    for i in range(nb_data):
        # Set the evidence
        inf_bn.setEvidence(data_records[i])
        # Run inference
        inf_bn.makeInference()
        # Compute posterior probability of target variable
        post_prob[i, :] = inf_bn.posterior(var_target).toarray()
        # Erase evidence
        inf_bn.eraseAllEvidence()
        if show_progress:
            sys.stdout.write("predict progress: {0:3.0%}\r".format(i/nb_data))

    post_prob_df = pd.DataFrame(post_prob,
                                index=data.index,
                                columns=bn.variable(var_target).labels())
    post_prob_df.columns.name = var_target
    return post_prob_df

In [17]:
# Chaque ligne des données fournies est utilisée comme évidence (conditionnement) pour calculer la loi de la variable cible sachant les données observées.

pred_prob = predict(bn, data=calls_dfd["a2"].loc[:10000, ["hour_a1", "hour_a2"]],
                    var_target="lead_answered_a2",
                    show_progress=True)

pred_var = pred_prob.idxmax(axis=1).rename("pred")
# On finit par évaluer le taux de prédictions du modèle par rapport aux données observées.

(calls_dfd["a2"].loc[:10000, "lead_answered_a2"] == pred_var).mean()

0.7949205079492051

In [18]:
# On finit par évaluer le taux de prédictions du modèle par rapport aux données observées.

(calls_dfd["a2"].loc[:10000, "lead_answered_a2"] == pred_var).mean()

0.7949205079492051

In [19]:
# Et la matrice de confusion associée :

pd.crosstab(calls_dfd["a2"].loc[:10000, "lead_answered_a2"], pred_var)

pred,no
lead_answered_a2,
no,7950
yes,2051
